In [ ]:
#%matplotlib inline  
import powerlaw
import csv
import glob
from collections import defaultdict
import gc
import sys
from multiprocessing import Pool
import pandas as pd

PUBYEAR_MIN = 1996
PUBYEAR_MAX = 2017

CITEYEAR_MAX = 2017

eid_info_dir = "../DATA/output/docinfo/eid_source_pubdate/"
eid_info_files = glob.glob(eid_info_dir + "*.txt")

pubyear_dict = {}
srcid_dict = {}
srcid_asjc = {}

_MAX_PROCESS = 4


def flatList(tdlist):
    templist = []
    for i in tdlist:
        templist += i
    return templist



In [ ]:
for nowfile in eid_info_files:
    with open(nowfile, 'rb') as csvfile: 
        reader = csv.reader(csvfile, delimiter='\t') 
        for row in reader: 
            if(row[1].isdigit()):
                pubyear_dict[row[0]] = int(row[1])
                srcid_dict[row[0]] = row[2]
gc.collect()

print len(pubyear_dict), len(srcid_dict)


In [ ]:
source_info_file = "../DATA/00_SCOPUS_SCIMAGO_MAPPING_DATA/src_final.txt"

with open(source_info_file, 'rb') as csvfile: 
    reader = csv.reader(csvfile, delimiter='\t') 
    for row in reader:
        srcid_asjc[row[0]] = row[5].split(";")
    print len(srcid_asjc.items())
gc.collect()


In [ ]:
PArrayLength = PUBYEAR_MAX - PUBYEAR_MIN + 1
CArrayLength = CITEYEAR_MAX - PUBYEAR_MIN + 1

network_info_dir = "../DATA/output/network/separated/"
network_info_files = glob.glob(network_info_dir + "*.txt")
print len(network_info_files)

SrcCountArray = []
for i in range(PArrayLength):
    temparray = []
    for j in range(CArrayLength):
        temparray.append({})
    SrcCountArray.append(temparray)

SrcPaperArray = []
for i in range(PArrayLength):
    SrcPaperArray.append({})    
    
gc.collect()
count_exist = 0
for k, v in pubyear_dict.iteritems():
    if(k not in srcid_dict):
        continue
    elif(srcid_dict[k] not in srcid_asjc):
        continue
    elif(v > PUBYEAR_MAX or v < PUBYEAR_MIN):
        continue
    SrcPaperArray[v - PUBYEAR_MIN][srcid_dict[k]] = (SrcPaperArray[v - PUBYEAR_MIN]).setdefault(srcid_dict[k], 0) + 1        
    for nC in range(v - PUBYEAR_MIN, CArrayLength):
        count_exist += 1
        SrcCountArray[v - PUBYEAR_MIN][nC][srcid_dict[k]] = 0

print "STEP2"
print count_exist
count_err = 0

for nowfile in network_info_files:
    with open(nowfile, 'rb') as csvfile: 
        reader = csv.reader(csvfile, delimiter='\t') 
        for row in reader: 
            if(row[0] not in srcid_dict or row[1] not in srcid_dict):
                pass
            elif(srcid_dict[row[0]] not in srcid_asjc or srcid_dict[row[1]] not in srcid_asjc):
                pass
            elif(pubyear_dict[row[0]] > PUBYEAR_MAX or pubyear_dict[row[0]] < PUBYEAR_MIN):
                pass
            elif(pubyear_dict[row[1]] > CITEYEAR_MAX):
                pass
            elif(pubyear_dict[row[1]] < pubyear_dict[row[0]]):
                count_err += 1 
            else:
                nP = pubyear_dict[row[0]] - PUBYEAR_MIN
                nC = pubyear_dict[row[1]] - PUBYEAR_MIN
                SrcCountArray[nP][nC][srcid_dict[row[0]]] += 1                
gc.collect()

print count_err
            

In [ ]:
count_data = pd.DataFrame(columns=["PubYear", "CiteYear", "SrcID", "Citecount", "Doccount"])
count_data

In [ ]:
temparray = []
for i in range(PArrayLength):
    for j in range(CArrayLength):
        temparray = []
        for key, val in SrcCountArray[i][j].iteritems():
            try:
                temp = [i, j, key, float(val), float(SrcPaperArray[i][key])]
            except ZeroDivisionError: 
                temp = [i, j, key, 0, 0]
            temparray.append(temp)
        count_data = count_data.append(pd.DataFrame(temparray, columns=["PubYear", "CiteYear", "SrcID", "Citecount", "Doccount"]), ignore_index = True)


In [ ]:
count_data['avg'] = count_data['Citecount']/count_data['Doccount']

In [ ]:
count_data